In [1]:
# [adapted from an IPython log file]
# This notebook hopes to perform a basic test on the newly-made
# pathfinder environment.

import time; print(time.asctime())
'Fri Oct  6 00:58:37 2017'

import sys
#sys.path.insert(0, '/home/usrnet/Software/installers/gym/')
sys.path.insert(0, '/home/usrnet/Work-Files/Path-Finder')
#sys.path.insert(0, '/home/usrnet/miniconda3/lib/python3.6/site-packages')
sys.path.insert(0, '/home/usrnet/Software/installers/gym/examples/agents')

import gym
from gym import envs, spaces
from gym.spaces import Discrete
from cleaner import *
from random_agent import *
from cem import *


Fri Oct  6 16:19:50 2017
Done.
current state:
-----------
----- -----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--o    @ --
--       --
-----I-----
-----------

current location: (5, 9)
 interface input >> 


In [2]:
try: reload(cleaner)
except: 
    import imp, cleaner; imp.reload(cleaner)
from cleaner import *

current state:
-----------
----- -----
--       --
-- o    @--
--    o  --
-        I-
--  o    --
--o    o --
--       --
----- -----
-----------

current location: (9, 5)
 interface input >> 


In [3]:
class MyEnv(gym.Env):
    def __init__(self, exp_env):
        self.action_space = Discrete(4)
        sz = exp_env.getGridSize()
        self.observation_space = spaces.Tuple((Discrete(sz[0]), \
                                Discrete(sz[1]), Discrete(4)))
        self.current_state = exp_env.get_random_starting_state()\
                                                    ['state']
        self.exp_env = exp_env        
        self.metadata = {'render.modes':['human','ansi','PRINT','NOPRINT']}
        self.reward_range = (0,1)
        
    def _reset(self): 
        self.current_state = self.exp_env.get_random_starting_state()['state']
        return self.current_state 
    
    def _render(self, mode=None, close=None): 
        p= print_state(self.current_state, \
                           'condensed', 'string_ret')
        if mode in ['human','PRINT']: 
            print(p)
            return p
        elif mode=='NOPRINT': 
            return p
        else: raise Exception(mode, 'render mode not defined')

    def sample(self): return random.choice(list(DVECS.keys()))

    def _step(self, actn):
        new_st, succ = self.exp_env.new_statem(\
            self.current_state, actn, valid_move_too=True)
        goalReached = self.exp_env.get_agent_loc(new_st) == \
                      self.exp_env.get_goal_loc(new_st)
        self.current_state = new_st
        return new_st, int(goalReached), (not succ) or goalReached, {}
    
   
my_env = MyEnv(ex)


In [6]:
for agent in [RandomAgent(my_env)]: 


    episode_count=100;
    reward=0;
    done=False

    for i in range(episode_count):
        if i%10: print ('episode '+str(i))
        ob = my_env.reset()
        if i%10: my_env.render(mode='PRINT')

        itr = 0
        done = False
        while not done:
            action = agent.act(ob, reward, done)
            if i%10: print('action #'+str(itr)+':', DIRVECS[DVECS[action]]); 
            itr += 1
            ob, reward, done, _ = my_env.step(action)
            if i%10: print('response: ',reward,done)
            if i%10: my_env.render(mode='PRINT')
            if i%10: 
                if input('continue')=='q': done=True
            if done and i%10: print('\t new trial:')







Adjusting 0 (5, 1) (5, 0)
episode 1
-----------
----- -----
--       --
-- @    o--
--    o  --
-I        -
--  o    --
--o    o --
--       --
----- -----
-----------

action #0: W
Adjusting 0 (1, 5) (0, 5)
response:  0 True
-----------
----- -----
--       --
-- @    o--
--    o  --
-I        -
--  o    --
--o    o --
--       --
----- -----
-----------

continue
	 new trial:
episode 2
-----------
-----I-----
--       --
-- o    o--
--    o  --
-         -
--  @    --
--o    o --
--       --
----- -----
-----------

action #0: W
Adjusting 0 (5, 1) (4, 1)
response:  0 True
-----------
-----I-----
--       --
-- o    o--
--    o  --
-         -
--  @    --
--o    o --
--       --
----- -----
-----------

continue
	 new trial:
episode 3
-----------
----- -----
--       --
-- o    o--
--    o  --
-         -
--  o    --
--@    o --
--       --
-----I-----
-----------

action #0: E
Adjusting 0 (5, 9) (6, 9)
response:  0 True
-----------
----- -----
--       --
-- o    o--
--    o  --
-   

KeyboardInterrupt: 